In [1]:
import tensorflow as tf
import numpy as np
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import matplotlib.pyplot as plt
import time
import os

In [28]:
# TODO: make env parent class
class env2048:  
    def __init__(self):
        self.driver = webdriver.Firefox()
        self.driver.implicitly_wait(3)
        self.driver.get('https://play2048.co')
        self.action_map = {0:'up', 1:'right', 2:'down', 3:'left'}
        
        # setup score memory for finding rewards
        self.score = 0
    
    # for this project, input will never be of magnitude 0 so I don't do a divide by 0 check
    def normalize(self, data):
        norm = np.linalg.norm(data)
        return data/norm
    
    def get_state(self, normalized = True):
        successful = False
        while not successful:
            successful = True
            tiles = self.driver.find_element_by_class_name('tile-container')
            values = [0]*16
            for tile in tiles.find_elements_by_tag_name('div'):
                className = tile.get_attribute('class')
                # lazy method of avoiding the tile-inner class
                if len(className) > 20:
                    # must do this to isolate the string w/ pos information
                    tile_pos = className.split(' ')[2]
                    # use indexes since size of string won't change so this is fast
                    x = int(tile_pos[14]) - 1
                    y = int(tile_pos[16]) - 1
                    index = 4*y + x
                    if tile.text != '':
                        values[index] = int(tile.text)
                    else:
                        # if here that means page hadn't fully updated
                        successful = False
        if normalized:
            return self.normalize(np.array([values]))
        else:
            return np.array([values])
    
    # while there is a score-addition tag that could make this more efficient
    # there were some issues where at times the text wouldn't be read properly
    # therefore I just manually calculate score addition for more consistent data
    def get_reward(self):
        useHighestBlock = False
        self.board_sum = True
        if useHighestBlock:
            return np.max(self.get_state(normalized = False))
        elif self.board_sum:
            return np.sum(self.get_state(normalized = False))
        else:
            r = self.driver.find_element_by_class_name('score-container')
            # do the split in case the score addition text is in the element
            score = int(r.text.split('\n')[0])
            r = score - self.score
            self.score = score
            return r
    
    def get_done(self):
        status = self.driver.find_element_by_class_name('game-message')
        return status.text != ""
    
    def take_action(self, action):
        body = self.driver.find_element_by_tag_name('body')
        if action == 0:
            body.send_keys(u'\ue013')
        elif action == 1:
            body.send_keys(u'\ue014')
        elif action == 2:
            body.send_keys(u'\ue015')
        elif action == 3:
            body.send_keys(u'\ue012')
        
        time.sleep(.1)
        
        return self.get_state(), self.get_reward(), self.get_done()
    
    def __del__(self):
        self.driver.close()

In [21]:
class DqnAgent(tf.keras.Model):
    def __init__(self, input_space_n, action_space_n):
        # setup model
        super(DqnAgent, self).__init__()
        self.INPUT_N = input_space_n
        self.ACTION_N = action_space_n
        self.h1 = tf.keras.layers.Dense(64, input_shape = (input_space_n,))
        self.prediction = tf.keras.layers.Dense(self.ACTION_N,  activation = tf.keras.activations.linear)
        
        # setup loss function
        self.loss = tf.keras.losses.MeanSquaredError()
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=.01)
    
    def call(self, x):
        # forward propogation
        x = self.h1(x)
        return self.prediction(x)
    
    
# experiences is numpy list s.t.
# 0 = state
# 1 = action
# 2 = next_state
# 3 = reward
# 4 = done


def train_step(model, experiences, repeat = 3, debug = False):
    DISCOUNT = .9
    if debug:
        print('experiences shape: ', experiences.shape)
    # get shapes to right shape
    states = np.array([e[0].reshape(16,) for e in experiences])
    
    # get q_vals/rewards setup and correct shape
    rewards = np.zeros((len(states) , 4))
    for i in range(states.shape[0]):
        e = experiences[i]
        if e[3] > 0:
            rewards[i,e[1]] = e[3]
        else:
            rewards[i,e[1]] = 0
        if not e[4] == True:
            rewards[i] += DISCOUNT * model(np.array([states[i]]).reshape(-1, 16))
    
    # normalize the rewards
    rewards = np.array(rewards)
    rewards = (rewards - np.mean(rewards, axis=0))/np.std(rewards, axis=0)
    print('mean of rewards, ', np.mean(rewards))
    
    # get sample
    batch_ratio = .1
    for r in range(repeat):
        if r % 10 == 0:
            print('iteration: ', r)
        batch_size = int(experiences.shape[0]*batch_ratio)
        select = np.random.randint(experiences.shape[0], size = batch_size)
        trainX = np.array(states[select]).reshape(-1, 16,)
        trainY = np.array(rewards[select]).reshape(-1, 4,)
        if debug:
            print('TrainX shape: ', trainX.shape)
            print('TrainY mean', np.mean(trainY))
            print('TrainY shape: ', trainY.shape)
            print(select[:10])
        with tf.GradientTape() as tape:
            predict = model(trainX)
            loss = model.loss(trainY, predict)
            if r % 10 == 0:
                print('Iteration', r, ', the loss is currently', loss.numpy())
        gradients = tape.gradient(loss, model.trainable_variables)
        model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

model = DqnAgent(16, 4)

In [22]:
e_greedy = False
def get_action(state):
    q_vals = model(state)[0]
    exp = np.exp(q_vals)
    softmax_vals = exp/np.sum(exp)
    #print(softmax_vals)
    EPSILON = .15
    if e_greedy:
        choice = np.random.choice([0,1], p=[EPSILON, 1- EPSILON])
        if choice == 1:
            try:
                return np.argmax(q_vals, axis = 0)
            except:
                print(softmax_vals)
                print(q_vals)
        else:
            return np.random.choice(4, p = [.25]*4)
    else:
        return np.random.choice(4, p= softmax_vals)

In [5]:
memo = []

In [6]:
scores = []

In [30]:
# train model
env = env2048()
num_episodes = 600
for episode in range(num_episodes):
    done = False
    state = env.get_state()
    experiences = []
    while not done:
        action = get_action(state)

        next_state, reward, done = env.take_action(action)

        experience = [state, action, next_state, reward, done]
        experiences.append(experience)
        state = next_state
     
    # cleanup experience
    scores.append(env.score)
    reward = env.get_reward()
    for experience in experiences:
        # punish uselss moves
        if (experience[0] == experience[2]).all():
            experience[3] = 0
        if env.board_sum: # give all experiences in this episode a rewards that is equal to sum of board
            experience[3] = reward
    
    # add experiences to memory
    memo.extend(experiences)
    data = np.array(memo)
    
    # occasionally train model on some of the collected data
    if (1+episode) % 5 == 0:
        train_step(model, data, repeat= 100)
        print(episode, 'th training episode')
    env.driver.find_element_by_class_name('restart-button').click()


mean of rewards,  2.5386270159162802e-15
iteration:  0
Iteration 0 , the loss is currently 1.0663928985595703
iteration:  10
Iteration 10 , the loss is currently 1.0371688604354858
iteration:  20
Iteration 20 , the loss is currently 1.032721757888794
iteration:  30
Iteration 30 , the loss is currently 1.0566681623458862
iteration:  40
Iteration 40 , the loss is currently 1.0347036123275757
iteration:  50
Iteration 50 , the loss is currently 1.035218358039856
iteration:  60
Iteration 60 , the loss is currently 1.0203419923782349
iteration:  70
Iteration 70 , the loss is currently 1.0382144451141357
iteration:  80
Iteration 80 , the loss is currently 1.0065908432006836
iteration:  90
Iteration 90 , the loss is currently 1.017730951309204
4 th training episode
mean of rewards,  -4.458467550436599e-15
iteration:  0
Iteration 0 , the loss is currently 1.0259482860565186
iteration:  10
Iteration 10 , the loss is currently 1.0323914289474487
iteration:  20
Iteration 20 , the loss is currently

iteration:  90
Iteration 90 , the loss is currently 1.019686222076416
54 th training episode
mean of rewards,  2.056034040826524e-15
iteration:  0
Iteration 0 , the loss is currently 1.0004160404205322
iteration:  10
Iteration 10 , the loss is currently 1.0028302669525146
iteration:  20
Iteration 20 , the loss is currently 1.0056594610214233
iteration:  30
Iteration 30 , the loss is currently 1.0061280727386475
iteration:  40
Iteration 40 , the loss is currently 1.0041742324829102
iteration:  50
Iteration 50 , the loss is currently 0.9975989460945129
iteration:  60
Iteration 60 , the loss is currently 1.0265319347381592
iteration:  70
Iteration 70 , the loss is currently 0.9889385104179382
iteration:  80
Iteration 80 , the loss is currently 1.0115809440612793
iteration:  90
Iteration 90 , the loss is currently 1.02306067943573
59 th training episode


W0817 01:08:29.523164 140228307916608 connectionpool.py:662] Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f89323e6cf8>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/73c3bbb3-59df-4edc-b48c-9fd1d407c9c0/window
W0817 01:08:29.524304 140228307916608 connectionpool.py:662] Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f893033d9b0>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/73c3bbb3-59df-4edc-b48c-9fd1d407c9c0/window
W0817 01:08:29.525345 140228307916608 connectionpool.py:662] Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f89281019e8>

W0817 01:08:29.554677 140228307916608 connectionpool.py:662] Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8928110c50>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/20a3da34-5837-4d9c-a04a-006b6d0ef6d5/window
W0817 01:08:29.555813 140228307916608 connectionpool.py:662] Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8928110a58>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/20a3da34-5837-4d9c-a04a-006b6d0ef6d5/window
Exception ignored in: <function env2048.__del__ at 0x7f89284aeea0>
Traceback (most recent call last):
  File "<ipython-input-20-8a21c0e81e54>", line 81, in __del__
  File "/home/brett/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/

mean of rewards,  1.7115428196039562e-15
iteration:  0
Iteration 0 , the loss is currently 1.0146466493606567
iteration:  10
Iteration 10 , the loss is currently 0.9840714931488037
iteration:  20
Iteration 20 , the loss is currently 0.9886392951011658
iteration:  30
Iteration 30 , the loss is currently 1.003871202468872
iteration:  40
Iteration 40 , the loss is currently 0.9963464140892029
iteration:  50
Iteration 50 , the loss is currently 1.009663701057434
iteration:  60
Iteration 60 , the loss is currently 0.9877459406852722
iteration:  70
Iteration 70 , the loss is currently 0.9986816644668579
iteration:  80
Iteration 80 , the loss is currently 0.9916666746139526
iteration:  90
Iteration 90 , the loss is currently 1.00174081325531
64 th training episode
mean of rewards,  3.111017559380731e-16
iteration:  0
Iteration 0 , the loss is currently 0.9961923360824585
iteration:  10
Iteration 10 , the loss is currently 0.9882507920265198
iteration:  20
Iteration 20 , the loss is currently 

iteration:  90
Iteration 90 , the loss is currently 0.9874922037124634
114 th training episode
mean of rewards,  1.6151352635817836e-15
iteration:  0
Iteration 0 , the loss is currently 0.9853821992874146
iteration:  10
Iteration 10 , the loss is currently 1.0109453201293945
iteration:  20
Iteration 20 , the loss is currently 0.9852930307388306
iteration:  30
Iteration 30 , the loss is currently 0.9902034401893616
iteration:  40
Iteration 40 , the loss is currently 0.983303427696228
iteration:  50
Iteration 50 , the loss is currently 1.0007520914077759
iteration:  60
Iteration 60 , the loss is currently 0.990890622138977
iteration:  70
Iteration 70 , the loss is currently 0.9988718628883362
iteration:  80
Iteration 80 , the loss is currently 0.999021053314209
iteration:  90
Iteration 90 , the loss is currently 0.9928194284439087
119 th training episode
mean of rewards,  -2.199882733779365e-15
iteration:  0
Iteration 0 , the loss is currently 0.9998800754547119
iteration:  10
Iteration 

StaleElementReferenceException: Message: The element reference of <div class="tile tile-8 tile-position-3-2"> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed


In [17]:
train_step(model, data, repeat= 300)
print('done training')

mean of rewards,  -3.917610456673311e-16
iteration:  0
Iteration 0 , the loss is currently 0.97614586353302
iteration:  10
Iteration 10 , the loss is currently 0.9879736304283142
iteration:  20
Iteration 20 , the loss is currently 0.9930416345596313
iteration:  30
Iteration 30 , the loss is currently 0.9863708019256592
iteration:  40
Iteration 40 , the loss is currently 0.977749228477478
iteration:  50
Iteration 50 , the loss is currently 0.9935055375099182
iteration:  60
Iteration 60 , the loss is currently 0.982075572013855
iteration:  70
Iteration 70 , the loss is currently 0.9909610748291016
iteration:  80
Iteration 80 , the loss is currently 0.9928528070449829
iteration:  90
Iteration 90 , the loss is currently 0.9759883880615234
iteration:  100
Iteration 100 , the loss is currently 1.0141304731369019
iteration:  110
Iteration 110 , the loss is currently 0.9921926856040955
iteration:  120
Iteration 120 , the loss is currently 0.9871148467063904
iteration:  130
Iteration 130 , the 

iteration:  1140
Iteration 1140 , the loss is currently 0.9995238780975342
iteration:  1150
Iteration 1150 , the loss is currently 1.0040082931518555
iteration:  1160
Iteration 1160 , the loss is currently 0.9946731328964233
iteration:  1170
Iteration 1170 , the loss is currently 0.9765210747718811
iteration:  1180
Iteration 1180 , the loss is currently 0.9822522401809692
iteration:  1190
Iteration 1190 , the loss is currently 0.9834169745445251
iteration:  1200
Iteration 1200 , the loss is currently 1.0011552572250366
iteration:  1210
Iteration 1210 , the loss is currently 0.9737172722816467
iteration:  1220
Iteration 1220 , the loss is currently 0.9755123853683472
iteration:  1230
Iteration 1230 , the loss is currently 0.98545902967453
iteration:  1240
Iteration 1240 , the loss is currently 0.9850686192512512
iteration:  1250
Iteration 1250 , the loss is currently 0.9848220944404602
iteration:  1260
Iteration 1260 , the loss is currently 1.0045324563980103
iteration:  1270
Iteration 

iteration:  2250
Iteration 2250 , the loss is currently 1.0005340576171875
iteration:  2260
Iteration 2260 , the loss is currently 0.9980844855308533
iteration:  2270
Iteration 2270 , the loss is currently 1.0030970573425293
iteration:  2280
Iteration 2280 , the loss is currently 0.9952865839004517
iteration:  2290
Iteration 2290 , the loss is currently 0.9883131384849548
iteration:  2300
Iteration 2300 , the loss is currently 0.9885216355323792
iteration:  2310
Iteration 2310 , the loss is currently 0.9821539521217346
iteration:  2320
Iteration 2320 , the loss is currently 0.9952273368835449
iteration:  2330
Iteration 2330 , the loss is currently 1.0068597793579102
iteration:  2340
Iteration 2340 , the loss is currently 1.0064204931259155
iteration:  2350
Iteration 2350 , the loss is currently 0.986929714679718
iteration:  2360
Iteration 2360 , the loss is currently 0.9917290210723877
iteration:  2370
Iteration 2370 , the loss is currently 0.9792357087135315
iteration:  2380
Iteration

iteration:  3360
Iteration 3360 , the loss is currently 0.9928527474403381
iteration:  3370
Iteration 3370 , the loss is currently 0.9926616549491882
iteration:  3380
Iteration 3380 , the loss is currently 0.9823941588401794
iteration:  3390
Iteration 3390 , the loss is currently 0.9912351965904236
iteration:  3400
Iteration 3400 , the loss is currently 1.0117264986038208
iteration:  3410
Iteration 3410 , the loss is currently 0.9964115023612976
iteration:  3420
Iteration 3420 , the loss is currently 0.9906454682350159
iteration:  3430
Iteration 3430 , the loss is currently 1.0032330751419067
iteration:  3440
Iteration 3440 , the loss is currently 0.9999409317970276
iteration:  3450
Iteration 3450 , the loss is currently 0.993975043296814
iteration:  3460
Iteration 3460 , the loss is currently 0.9714041352272034
iteration:  3470
Iteration 3470 , the loss is currently 0.9740735292434692
iteration:  3480
Iteration 3480 , the loss is currently 1.0092452764511108
iteration:  3490
Iteration

iteration:  4470
Iteration 4470 , the loss is currently 1.0195415019989014
iteration:  4480
Iteration 4480 , the loss is currently 0.9794303178787231
iteration:  4490
Iteration 4490 , the loss is currently 0.9861177206039429
iteration:  4500
Iteration 4500 , the loss is currently 0.9904816746711731
iteration:  4510
Iteration 4510 , the loss is currently 0.990121066570282
iteration:  4520
Iteration 4520 , the loss is currently 0.9908421635627747
iteration:  4530
Iteration 4530 , the loss is currently 0.9721967577934265
iteration:  4540
Iteration 4540 , the loss is currently 1.0068144798278809
iteration:  4550
Iteration 4550 , the loss is currently 1.0056573152542114
iteration:  4560
Iteration 4560 , the loss is currently 0.992931067943573
iteration:  4570
Iteration 4570 , the loss is currently 0.9811499118804932
iteration:  4580
Iteration 4580 , the loss is currently 0.9841657280921936
iteration:  4590
Iteration 4590 , the loss is currently 1.0058313608169556
iteration:  4600
Iteration 

...

iteration:  25140
Iteration 25140 , the loss is currently 0.9725174307823181
iteration:  25150
Iteration 25150 , the loss is currently 1.009662389755249
iteration:  25160
Iteration 25160 , the loss is currently 1.0090049505233765
iteration:  25170
Iteration 25170 , the loss is currently 0.9961330890655518
iteration:  25180
Iteration 25180 , the loss is currently 0.9922797679901123
iteration:  25190
Iteration 25190 , the loss is currently 1.0038747787475586
iteration:  25200
Iteration 25200 , the loss is currently 0.9941167235374451
iteration:  25210
Iteration 25210 , the loss is currently 0.9965454936027527
iteration:  25220
Iteration 25220 , the loss is currently 1.005096435546875
iteration:  25230
Iteration 25230 , the loss is currently 0.9851616621017456
iteration:  25240
Iteration 25240 , the loss is currently 0.986116886138916
iteration:  25250
Iteration 25250 , the loss is currently 0.9888976812362671
iteration:  25260
Iteration 25260 , the loss is currently 0.9836450815200806
it

iteration:  26220
Iteration 26220 , the loss is currently 0.9644482135772705
iteration:  26230
Iteration 26230 , the loss is currently 0.978066086769104
iteration:  26240
Iteration 26240 , the loss is currently 1.0037972927093506
iteration:  26250
Iteration 26250 , the loss is currently 0.98975670337677
iteration:  26260
Iteration 26260 , the loss is currently 1.0033060312271118
iteration:  26270
Iteration 26270 , the loss is currently 0.9817700982093811
iteration:  26280
Iteration 26280 , the loss is currently 0.9944252967834473
iteration:  26290
Iteration 26290 , the loss is currently 0.9970530271530151
iteration:  26300
Iteration 26300 , the loss is currently 0.9894599914550781
iteration:  26310
Iteration 26310 , the loss is currently 0.9933525919914246
iteration:  26320
Iteration 26320 , the loss is currently 0.9817519187927246
iteration:  26330
Iteration 26330 , the loss is currently 0.9889774322509766
iteration:  26340
Iteration 26340 , the loss is currently 0.987904965877533
ite

iteration:  27300
Iteration 27300 , the loss is currently 0.9924706220626831
iteration:  27310
Iteration 27310 , the loss is currently 0.9809175133705139
iteration:  27320
Iteration 27320 , the loss is currently 0.9852770566940308
iteration:  27330
Iteration 27330 , the loss is currently 0.997734546661377
iteration:  27340
Iteration 27340 , the loss is currently 0.993989109992981
iteration:  27350
Iteration 27350 , the loss is currently 0.9727087616920471
iteration:  27360
Iteration 27360 , the loss is currently 0.9816675186157227
iteration:  27370
Iteration 27370 , the loss is currently 0.9995225667953491
iteration:  27380
Iteration 27380 , the loss is currently 0.9930263161659241
iteration:  27390
Iteration 27390 , the loss is currently 0.9899560809135437
iteration:  27400
Iteration 27400 , the loss is currently 0.9927341938018799
iteration:  27410
Iteration 27410 , the loss is currently 0.987888753414154
iteration:  27420
Iteration 27420 , the loss is currently 0.9980634450912476
it

iteration:  28380
Iteration 28380 , the loss is currently 0.9775968790054321
iteration:  28390
Iteration 28390 , the loss is currently 0.9797887206077576
iteration:  28400
Iteration 28400 , the loss is currently 1.012380599975586
iteration:  28410
Iteration 28410 , the loss is currently 0.9885318279266357
iteration:  28420
Iteration 28420 , the loss is currently 1.0035251379013062
iteration:  28430
Iteration 28430 , the loss is currently 0.9961012005805969
iteration:  28440
Iteration 28440 , the loss is currently 0.9831315875053406
iteration:  28450
Iteration 28450 , the loss is currently 1.0010151863098145
iteration:  28460
Iteration 28460 , the loss is currently 0.9960935115814209
iteration:  28470
Iteration 28470 , the loss is currently 0.9845823645591736
iteration:  28480
Iteration 28480 , the loss is currently 1.0015654563903809
iteration:  28490
Iteration 28490 , the loss is currently 0.9703657627105713
iteration:  28500
Iteration 28500 , the loss is currently 0.9892054796218872


iteration:  29460
Iteration 29460 , the loss is currently 0.9806605577468872
iteration:  29470
Iteration 29470 , the loss is currently 0.9830146431922913
iteration:  29480
Iteration 29480 , the loss is currently 0.9805755615234375
iteration:  29490
Iteration 29490 , the loss is currently 0.9895049333572388
iteration:  29500
Iteration 29500 , the loss is currently 0.9602532386779785
iteration:  29510
Iteration 29510 , the loss is currently 1.019428014755249
iteration:  29520
Iteration 29520 , the loss is currently 0.9868649840354919
iteration:  29530
Iteration 29530 , the loss is currently 0.9815470576286316
iteration:  29540
Iteration 29540 , the loss is currently 1.0023115873336792
iteration:  29550
Iteration 29550 , the loss is currently 1.023728370666504
iteration:  29560
Iteration 29560 , the loss is currently 1.004820704460144
iteration:  29570
Iteration 29570 , the loss is currently 1.0099045038223267
iteration:  29580
Iteration 29580 , the loss is currently 1.0027841329574585
it

In [14]:
def visualize(state):
    for x in range(16):
        if x % 4 == 0 and not x == 0:
            print()
        print(state[x], end=' ')


In [61]:
e = env2048()

In [62]:
s = e.get_state(normalized=False)

In [31]:
s = memo[3000][0]
visualize(s.reshape(16,))
print()
print(memo[3000][1])
print(model(s))
get_action(s)

NameError: name 'visualize' is not defined

In [39]:
np.save('modelWeights150E', np.array(model.weights))